In [1]:
from wrapper import *
import MahjongPy as mp
import os
import time
import logging
import warnings

import numpy as np
import scipy.io as sio
import torch
import matplotlib.pyplot as plt

import mahjong
from mahjong.shanten import Shanten

from models import VLOG

In [2]:

player_i_hand_start_ind = [0, 63, 69, 75]  # later 3 in oracle_obs
player_i_side_start_ind = [6, 12, 18, 24]
player_i_river_start_ind = [30, 37, 44, 51]

dora_indicator_ind = 58
dora_ind = 59
game_wind_ind = 60
self_wind_ind = 61
latest_tile_ind = 62

aka_tile_ints = [16, 16 + 36, 16 + 36 + 36]
player_obs_width = 63

CHILEFT = 34
CHIMIDDLE = 35
CHIRIGHT = 36
PON = 37
ANKAN = 38
MINKAN = 39
ADDKAN = 40

RIICHI = 41
RON = 42
TSUMO = 43
PUSH = 44
PASS_RESPONSE = 45
PASS_RIICHI = 46

In [3]:

UNICODE_TILES = """
    🀇 🀈 🀉 🀊 🀋 🀌 🀍 🀎 🀏 
    🀙 🀚 🀛 🀜 🀝 🀞 🀟 🀠 🀡
    🀐 🀑 🀒 🀓 🀔 🀕 🀖 🀗 🀘
    🀀 🀁 🀂 🀃
    🀆 🀅 🀄
""".split()

EXPLAINS = UNICODE_TILES + ["Chi-Left", "Chi-Middle", "Chi-Right", "Pon", "An-Kan",
           "Min-Kan", "Add-Kan", "Riichi", "Ron", "Tsumo", "Push 99", "Pass", "Pass"]

shanten = Shanten()

def to_unicode_tails(tiles, sort=True):
    unicode_tail_string = ""
    if sort:
        tiles = np.sort(np.array(tiles))

    for tile in tiles:
        unicode_tail_string = unicode_tail_string + UNICODE_TILES[int(tile / 4)]

    return unicode_tail_string

In [4]:
env_test = EnvMahjong3(force_riichi=False, force_win=False, append_aval_action_obs=True)
max_steps = 1000

In [5]:
# agent_test0 = torch.load("C:/code/snake/amlt/vlog_offlinemj_20210724/search_envid_1_beta_1e-05_alpha_1_kldt_10_cqla_10/data/mahjong_VLOG_DDQN_0.model")
# agent_test1 = torch.load("C:/code/snake/amlt/vlog_offlinemj_20210724/search_envid_1_beta_1e-05_alpha_1_kldt_10_cqla_10/data/mahjong_VLOG_DDQN_1.model")
# agent_test2 = torch.load("C:/code/snake/amlt/vlog_offlinemj_20210724/search_envid_1_beta_1e-05_alpha_1_kldt_10_cqla_10/data/mahjong_VLOG_DDQN_2.model")
# agent_test = torch.load("./vlog_offlinemj_20210724/search_envid_1_beta_0_alpha_1_kldt_-1_cqla_100/data/mahjong_VLOG_DDQN_0.model")


if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"


# agent_test0 = torch.load("./vlog_offlinemj_bc_20210802/search_beta_1e-06_alpha_1_kldt_10_bnt_0_eqn_1_cqla_0/data/mahjong_VLOG_BC_0.model", map_location=torch.device(device))
# agent_test1 = torch.load("./vlog_offlinemj_bc_20210802/search_beta_1e-06_alpha_1_kldt_10_bnt_0_eqn_1_cqla_0/data/mahjong_VLOG_BC_1.model", map_location=torch.device(device))
# agent_test2 = torch.load("./vlog_offlinemj_bc_20210802/search_beta_1e-06_alpha_1_kldt_10_bnt_0_eqn_1_cqla_0/data/mahjong_VLOG_BC_0.model", map_location=torch.device(device))
# agent_test3 = torch.load("./vlog_offlinemj_bc_20210802/search_beta_1e-06_alpha_1_kldt_10_bnt_0_eqn_1_cqla_0/data/mahjong_VLOG_BC_1.model", map_location=torch.device(device))

# agent_test0 = torch.load("./vlog_offlinemj_bc_20210802/search_beta_1e-06_alpha_1_kldt_10_bnt_0_eqn_1_cqla_0/data/mahjong_VLOG_BC_0.model", map_location=torch.device(device))
# agent_test1 = torch.load("./vlog_offlinemj_bc_20210802/search_beta_1e-06_alpha_1_kldt_10_bnt_0_eqn_1_cqla_0/data/mahjong_VLOG_BC_1.model", map_location=torch.device(device))
# agent_test2 = torch.load("./vlog_offlinemj_bc_20210802/search_beta_0_alpha_1_kldt_-1_bnt_0_eqn_1_cqla_0/data/mahjong_VLOG_BC_2.model", map_location=torch.device(device))
# agent_test3 = torch.load("./vlog_offlinemj_bc_20210802/search_beta_0_alpha_1_kldt_-1_bnt_0_eqn_1_cqla_0/data/mahjong_VLOG_BC_3.model", map_location=torch.device(device))


agent_test0 = torch.load("./vlog_offlinemj_cql_20210808/search_envid_1_beta_0_alpha_1_kldt_-1_bnt_0_dpo_0_eqn_0_cqla_300/data/mahjong_VLOG_DDQN_3.model", map_location=torch.device(device))
agent_test1 = torch.load("./vlog_offlinemj_cql_20210808/search_envid_1_beta_0_alpha_1_kldt_-1_bnt_0_dpo_0_eqn_0_cqla_300/data/mahjong_VLOG_DDQN_3.model", map_location=torch.device(device))
agent_test2 = torch.load("./vlog_offlinemj_cql_20210809/search_envid_1_beta_0_alpha_1_kldt_-1_bnt_0_dpo_0_eqn_0_cqla_5/data/mahjong_VLOG_DDQN_2.model", map_location=torch.device(device))
agent_test3 = torch.load("./vlog_offlinemj_cql_20210809/search_envid_1_beta_1e-08_alpha_1_kldt_30_bnt_0_dpo_0_eqn_0_cqla_5/data/mahjong_VLOG_DDQN_2.model", map_location=torch.device(device))



In [6]:

agents_test = [agent_test0, agent_test1, agent_test2, agent_test3]

for agent in agents_test:
    agent.device = device


In [15]:
full_obs = False

EpiTestRet = 0
steps_taken = 0

tsumo_times = np.array([0, 0, 0, 0])
liuju_times = 0
noten_times = 0
houjyu_times = np.array([0, 0, 0, 0])
riichi_times = np.array([0, 0, 0, 0])

riichi_agari_times = np.array([0, 0, 0, 0])
yami_agari_times = np.array([0, 0, 0, 0])

not_riichi_houjyu_times = np.array([0, 0, 0, 0])

fulu_times = np.array([0, 0, 0, 0])
agari_turns = np.array([0, 0, 0, 0])
agari_times = np.array([0, 0, 0, 0])

agari_scores = np.array([0, 0, 0, 0])
total_results = []

game = 0


In [16]:
import numpy

In [ ]:

while game < 200000:
    print("==============================Game {}=============================".format(game))
    # reset each episode
    if game % 10 == 1:
        try:
            print("------------------------- Statistics ----------------------")
            print("平均结果:", (np.mean(np.array(total_results), axis=0)))
            print("结果M.S.E.", np.array(total_results).std(axis=0) / np.sqrt(game))
            print("自摸率：", numpy.array2string(tsumo_times * 100 / agari_times, precision=3, separator=',    '))
            print("放铳率：", numpy.array2string(houjyu_times * 100 / game, precision=3, separator=',    '))
            print("和了率：", numpy.array2string(agari_times * 100 / game, precision=3, separator=',    '))
            print("副露率：", numpy.array2string(fulu_times * 100 / game, precision=3, separator=',    '))
            print("立直率：", numpy.array2string(riichi_times * 100 / game, precision=3, separator=',    '))
            print("立直成功率：", numpy.array2string(riichi_agari_times * 100 / riichi_times, precision=3, separator=',    '))
            print("暗和率：", numpy.array2string(yami_agari_times  * 100/ agari_times, precision=3, separator=',    '))
            print("流局率：", numpy.array2string((game - np.sum(agari_times)) * 100 / game, precision=3, separator=',    '))
            print("平均和牌时还剩几张牌：", numpy.array2string(agari_turns / agari_times, precision=3, separator=',    '))
            print("没立直时的放铳率：", numpy.array2string(not_riichi_houjyu_times * 100 / (game - riichi_times), precision=3, separator=',    '))
            print("四家无听流局率：", noten_times * 100 / game)
            print("平均打点：", numpy.array2string(agari_scores / agari_times, precision=3, separator=',    '))
        except:
            pass
    try:
        sp = env_test.reset(game % 4, 'east')
        result_array = np.zeros([4])
        done = False
        has_riichi = [False] * 4

        for tt in range(max_steps):

            # if curr_pid != 0:
            #     a = valid_actions[np.random.randint(len(valid_actions))]

            valid_actions = env_test.get_valid_actions(nhot=False)

            if len(valid_actions) == 1:
                env_test.t.make_selection(0)

            else:
                curr_pid = env_test.get_curr_player_id()

                action_mask = env_test.get_valid_actions(nhot=True)

                if not full_obs:
                    a = agents_test[curr_pid].select(env_test.get_obs(curr_pid), action_mask, greedy=True)
                else:
                    a = agents_test[curr_pid].select(env_test.get_full_obs(curr_pid), action_mask, greedy=True)
                
                if a == RIICHI:
                    riichi_times[curr_pid] += 1
                    has_riichi[curr_pid] = True

#                 if a == TSUMO:
#                     print(env_test.get_full_obs(0)[player_i_river_start_ind[curr_pid]:player_i_river_start_ind[curr_pid] + 7,  int(env_test.latest_tile / 4)])
#                     plt.pcolor(env_test.get_obs(curr_pid))
#                     plt.show()

#                 if curr_pid == 0 and len(valid_actions) > 1:
#                     print("-------------- Step {}, player {} ----------------".format(tt, curr_pid))
#                     print(env_test.Phases[env_test.t.get_phase()], "Recent Tile:", to_unicode_tails([env_test.latest_tile]))
#                     side_tiles_0 = [st[0] for st in env_test.side_tiles[curr_pid]]
#                     print("手牌: ", to_unicode_tails(env_test.hand_tiles[curr_pid]),
#                           "； 副露：", to_unicode_tails(side_tiles_0))
#                     if a < 34:
#                         agent_selection_str = UNICODE_TILES[a]
#                     else:
#                         agent_selection_str = EXPLAINS[a]
                
#                     print("Agent选择打: ", agent_selection_str)

                sp, r, done, _ = env_test.step(curr_pid, a)
                if r < 0 and not done:
                    result_array[curr_pid] += r * 100

            steps_taken += 1

            if done or env_test.t.get_phase() == 16:
                
                payoffs = env_test.get_payoffs()
                # EpiTestRet += payoffs[0]
                result_array += np.array(payoffs) * 100
                print("~~~~~~~~~~~~Result: ", env_test.t.get_result().result_type, payoffs)
#                 print("remaining tiles:", env_test.t.get_remain_tile())
                total_results.append(result_array)
                for uid in range(4):
                    if len(env_test.side_tiles[uid]) >= 3:
                        fulu_times[uid] += 1
                
                if env_test.t.get_remain_tile() == 0:
                    liuju_times += 1
                    if np.max(payoffs) == 0:
                        noten_times += 1
                
                if env_test.t.get_remain_tile() > 0 and np.max(payoffs) > 0:  # not 流局
                    win_uid = np.argmax(payoffs)
                    agari_turns[win_uid] += env_test.t.get_remain_tile()
                    if has_riichi[win_uid]:
                        riichi_agari_times[win_uid] += 1
                    else:
                        yami_agari_times[win_uid] += 1
                    
                    if np.count_nonzero(result_array < 0) == 3:
                        tsumo_times[win_uid] += 1
                    else:
                        houjyu_times[np.argmin(result_array)] += 1
                        if not has_riichi[np.argmin(result_array)]:
                            not_riichi_houjyu_times[np.argmin(result_array)] += 1
                        
                    agari_times[win_uid] += 1
                    agari_scores[win_uid] += np.max(result_array)
                    
#                     win_pid = np.argmax(result_array)
#                     print(env_test.Phases[env_test.t.get_phase()], "Recent Tile:", to_unicode_tails([env_test.latest_tile]))
#                     side_tiles_0 = [st[0] for st in env_test.side_tiles[win_pid]]
#                     print("手牌: ", to_unicode_tails(env_test.hand_tiles[win_pid]),
#                           "； 副露：", to_unicode_tails(side_tiles_0))

                # print(env_test.t.game_log.to_string())
#                 if np.max(payoffs) > 0:
#                     plt.pcolor(sp)
#                     plt.show()
#                     print(env_test.Phases[env_test.t.get_phase()], "Recent Tile:", to_unicode_tails([env_test.latest_tile]))
#                     side_tiles_0 = [st[0] for st in env_test.side_tiles[np.argmax(payoffs)]]
#                     print("手牌: ", to_unicode_tails(env_test.hand_tiles[np.argmax(payoffs)]),
#                       "； 副露：", to_unicode_tails(side_tiles_0))
                break

#             if not done and env_test.t.get_phase() == 0:
#                 shanten_num = np.zeros(4)
#                 for pid in range(4):
#                     shanten_num[pid] = shanten.calculate_shanten(TilesConverter.to_34_array(
#                         env_test.hand_tiles[pid] + [st[0] for st in env_test.side_tiles[pid]]))
#                 print("向听数:", shanten_num)
        game += 1

    except Exception as e:
        print(e)
        if a < 34:
            agent_selection_str = UNICODE_TILES[a]
        else:
            agent_selection_str = EXPLAINS[a]
        print("Agent选择打: ", a)
        time.sleep(0.1)
        continue


==============================Game 629=============================
~~~~~~~~~~~~Result:  ResultType.RonAgari [-13.0, 0.0, 0.0, 13.0]
==============================Game 630=============================
~~~~~~~~~~~~Result:  ResultType.TsumoAgari [-3.0, 11.0, -5.0, -3.0]
==============================Game 631=============================
------------------------- Statistics ----------------------
平均结果: [-129.63549921 -102.06022187    0.63391442   34.54833597]
结果M.S.E. [125.72054742 119.58518045 129.79155655 139.78394113]
自摸率： [51.754,    54.622,    53.6  ,    57.944]
放铳率： [7.607,    8.399,    8.716,    8.875]
和了率： [18.067,    18.859,    19.81 ,    16.957]
副露率： [29.002,    25.04 ,    34.865,    27.892]
立直率： [19.334,    20.919,    15.689,    11.252]
立直成功率： [29.508,    22.727,    46.465,    46.479]
暗和率： [68.421,    74.79 ,    63.2  ,    69.159]
流局率： 26.307
平均和牌时还剩几张牌： [23.404,    24.807,    25.576,    22.991]
没立直时的放铳率： [7.269,    8.417,    9.774,    9.464]
四家无听流局率： 1.4263074484944533
平均打点： [

In [ ]:
env_test.t.get_phase()

In [ ]:
mp.ResultType.TsumoAgari

In [ ]:
total_results_array = np.array(total_results)
sampled_games = np.zeros([10000, 8, 4])
for i in range(10000):
    sampled_games[i] = total_results_array[np.random.choice(len(total_results_array), 8)]
sampled_match_results = sampled_games.sum(-2)

win_rate = ((sampled_games[:, :, 0] + sampled_games[:, :, 1]).sum(1) > (sampled_games[:, :, 2] + sampled_games[:, :, 3]).sum(1)).mean(0) 
print(win_rate)

In [ ]:
for pid in range(4):
    print(np.mean(np.argmax(sampled_match_results, axis=-1) == pid) * 100, np.std(np.argmax(sampled_match_results, axis=-1) == pid) / 10000 ** 0.5 * 100)

In [ ]:
np.array(total_results).std(axis=0) * 100 / np.sqrt(game)

In [ ]:
to_unicode_tails(env_test.hand_tiles[3])

In [ ]:
plt.pcolor(env_test.get_obs(3)[:6, :])
plt.show()

In [ ]:
env_test.can_riichi_tiles_id

In [ ]:
for act in env_test.t.get_self_actions():
    print(act.action)
    print(int(act.correspond_tiles[0].tile))

In [ ]:
env_test.riichi_tile_id

In [ ]:
int(env_test.t.get_selected_action_tile().tile)

In [ ]:
env_test.t.get_response_actions()[0].action

In [ ]:
env_test.t.players[curr_pid].hand.__len__()

In [ ]:
env_test.Phases[env_test.t.get_phase()]

In [ ]:
env_test.hand_tiles[curr_pid]

In [ ]:
env_test.can_riichi

In [ ]:
env_test.t.get_self_actions()[0].action

In [ ]:
a = set([1,2,3,4,4])

In [ ]:
list(a)

In [ ]:
mp.Action.Kakan